In [1]:
!git clone https://github.com/lessw2020/Ranger-Deep-Learning-Optimizer.git
!pip install adamp

Cloning into 'Ranger-Deep-Learning-Optimizer'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 135 (delta 34), reused 17 (delta 6), pack-reused 69
Receiving objects: 100% (135/135), 181.12 KiB | 1.02 MiB/s, done.
Resolving deltas: 100% (59/59), done.
  Created wheel for adamp: filename=adamp-0.3.0-py3-none-any.whl size=5998 sha256=73b6d6525fda88d547418858dbd57e32939a22e9741f311a99e6072d82e1cc45
  Stored in directory: /root/.cache/pip/wheels/bb/95/21/ced2d2cb9944e3a72e58fece7958973eed3fd8d0aeb6e2e450
Successfully built adamp
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
sys.path.append('./Ranger-Deep-Learning-Optimizer/ranger')
# sys.setrecursionlimit(10**6)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm.autonotebook import tqdm
from pprint import pprint
import cv2, glob, time, random, os, warnings
warnings.filterwarnings('ignore')

import timm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
from torch.optim import Adam, AdamW, SGD

import albumentations as A
from albumentations.pytorch import ToTensorV2

from adamp import AdamP
# https://github.com/lessw2020/Ranger-Deep-Learning-Optimizer
# from ranger import Ranger  # this is from ranger.py
from ranger913A import RangerVA  # this is from ranger913A.py
from rangerqh import RangerQH  # this is from rangerqh.py
from ranger2020 import Ranger

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """


In [4]:
# get the list of pretrained models
model_names = timm.list_models()
pprint(model_names)

['adv_inception_v3',
 'cspdarknet53',
 'cspdarknet53_iabn',
 'cspresnet50',
 'cspresnet50d',
 'cspresnet50w',
 'cspresnext50',
 'cspresnext50_iabn',
 'darknet53',
 'densenet121',
 'densenet121d',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenet264',
 'densenet264d_iabn',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2next',
 'dla60x',
 'dla60x_c',
 'dla102',
 'dla102x',
 'dla102x2',
 'dla169',
 'dpn68',
 'dpn68b',
 'dpn92',
 'dpn98',
 'dpn107',
 'dpn131',
 'eca_vovnet39b',
 'ecaresnet18',
 'ecaresnet50',
 'ecaresnet50d',
 'ecaresnet50d_pruned',
 'ecaresnet101d',
 'ecaresnet101d_pruned',
 'ecaresnetlight',
 'ecaresnext26tn_32x4d',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b2a',
 'efficientnet_b3',
 'efficientnet_b3_pruned',
 'efficientnet_b3a',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'efficientnet_b8',


## CFG

In [19]:
BATCH_SIZE = 16
EPOCHS = 3
IMG_SIZE = 512
ITER_FREQ = 500
NUM_WORKERS = 8
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
SEED = 1111

LR = 1e-4
MIN_LR = 5e-5 # SAM, CosineAnnealingWarmRestarts
WEIGHT_DECAY = 1e-6
MOMENTUM = 0.9
T_0 = EPOCHS # SAM, CosineAnnealingWarmRestarts
MAX_NORM = 1000

SNAPMIX = True
SNAPMIX_ALPHA = 5.0
SNAPMIX_PCT = 0.5

MODEL_ARCH = 'tf_efficientnet_b3_ns'
ITERS_TO_ACCUMULATE = 1

CUTMIX = False
CM_START = 0
CM_ALPHA = 1

BASE_OPTIMIZER = SGD #for SAM, Ranger
OPTIMIZER = 'AdamW'
if OPTIMIZER == 'SAM':
    SAM = True

SCHEDULER = 'CosineAnnealingWarmRestarts'
SCHEDULER_UPDATE = 'epoch'

CRITERION = 'CrossEntropyLoss'
LABEL_SMOOTH = 0.15
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
class AverageMeter(object):
    
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.sum = 0
        self.avg = 0
        self.count = 0
        
    def update(self, val, n=1):
        self.val = val
        self.sum += val*n
        self.count += n
        self.avg = self.sum / self.count

def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(SEED)

In [21]:
TRAIN_DIR = '../input/cassava-leaf-disease-merged/train/'
df = pd.read_csv('../input/cassava-leaf-disease-merged/merged.csv')
# df

In [22]:
class CLD_train_DS(Dataset):
    
    def __init__(self, df, transform=None):
        self.df = df
        self.image_id = df['image_id'].values
        self.label = df['label'].values
        self.transform = transform
        
    def __len__(self) -> int:
        return len(self.df)
        
    def __getitem__(self, idx):
        
        image_id = self.image_id[idx]
        img = cv2.imread(TRAIN_DIR + image_id)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img = img / 255.0
        
        if self.transform:
            trans_img = self.transform(image=img)
            img = trans_img['image']
            
        label = torch.tensor(self.label[idx]).long()
        
        return img, label

In [23]:
def get_transform(*, train=True):
    
    if train:
        return A.Compose([
            A.RandomResizedCrop(IMG_SIZE, IMG_SIZE),
            A.Transpose(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.Normalize(mean=MEAN, std=STD),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.CenterCrop(IMG_SIZE, IMG_SIZE),
            A.Resize(IMG_SIZE, IMG_SIZE),
            A.Normalize(mean=MEAN, std=STD),
            ToTensorV2(),
        ])

SAM optimizer

In [24]:
class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

## Loss Function

In [25]:
class LabelSmoothingLoss(nn.Module): 
    def __init__(self, classes=5, smoothing=0.0, dim=-1): 
        super(LabelSmoothingLoss, self).__init__() 
        self.confidence = 1.0 - smoothing 
        self.smoothing = smoothing 
        self.cls = classes 
        self.dim = dim 
    def forward(self, pred, target): 
        if CRITERION == 'LabelSmoothingLoss':
            pred = pred.log_softmax(dim=self.dim) 
        with torch.no_grad():
            true_dist = torch.zeros_like(pred) 
            true_dist.fill_(self.smoothing / (self.cls - 1)) 
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence) 
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))
    
class TaylorSoftmax(nn.Module):
    '''
    This is the autograd version
    '''
    def __init__(self, dim=1, n=2):
        super(TaylorSoftmax, self).__init__()
        assert n % 2 == 0
        self.dim = dim
        self.n = n

    def forward(self, x):
        '''
        usage similar to nn.Softmax:
            >>> mod = TaylorSoftmax(dim=1, n=4)
            >>> inten = torch.randn(1, 32, 64, 64)
            >>> out = mod(inten)
        '''
        fn = torch.ones_like(x)
        denor = 1.
        for i in range(1, self.n+1):
            denor *= i
            fn = fn + x.pow(i) / denor
        out = fn / fn.sum(dim=self.dim, keepdims=True)
        return out
    
class TaylorCrossEntropyLoss(nn.Module):
    '''
    This is the autograd version
    '''
    def __init__(self, n=2, ignore_index=-1, reduction='mean'):
        super(TaylorCrossEntropyLoss, self).__init__()
        assert n % 2 == 0
        self.taylor_softmax = TaylorSoftmax(dim=1, n=n)
        self.reduction = reduction
        self.ignore_index = ignore_index

    def forward(self, logits, labels):
        '''
        usage similar to nn.CrossEntropyLoss:
            >>> crit = TaylorCrossEntropyLoss(n=4)
            >>> inten = torch.randn(1, 10, 64, 64)
            >>> label = torch.randint(0, 10, (1, 64, 64))
            >>> out = crit(inten, label)
        '''
        log_probs = self.taylor_softmax(logits).log()
        loss = F.nll_loss(log_probs, labels, reduction=self.reduction,
                ignore_index=self.ignore_index)
        return loss
    
class TaylorSmoothedLoss(nn.Module):

    def __init__(self, n=2, ignore_index=-1, reduction='mean', smoothing=0.2):
        super(TaylorSmoothedLoss, self).__init__()
        assert n % 2 == 0
        self.taylor_softmax = TaylorSoftmax(dim=1, n=n)
        self.reduction = reduction
        self.ignore_index = ignore_index
        self.lab_smooth = LabelSmoothingLoss(5, smoothing=LABEL_SMOOTH)

    def forward(self, logits, labels):

        log_probs = self.taylor_softmax(logits).log()
        #loss = F.nll_loss(log_probs, labels, reduction=self.reduction,
        #        ignore_index=self.ignore_index)
        loss = self.lab_smooth(log_probs, labels)
        return loss
    
class CutMixCriterion(nn.Module):
    def __init__(self, criterion):
        super(CutMixCriterion, self).__init__()
        self.criterion = criterion

    def forward(self, preds, labels):
        targets1 = labels[:,0] 
        targets2 = labels[:,1]
        lam = labels[0,2]
        return lam * self.criterion(
            preds, targets1) + (1 - lam) * self.criterion(preds, targets2)

## Augmentations

SnapMix

In [26]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def get_spm(input,target,model):
    imgsize = (IMG_SIZE, IMG_SIZE)
    bs = input.size(0)
    with torch.no_grad():
        output,fms = model(input)
        clsw = model.classifier
        weight = clsw.weight.data
        bias = clsw.bias.data
        weight = weight.view(weight.size(0),weight.size(1),1,1)
        fms = F.relu(fms)
        poolfea = F.adaptive_avg_pool2d(fms,(1,1)).squeeze()
        clslogit = F.softmax(clsw.forward(poolfea))
        logitlist = []
        for i in range(bs):
            logitlist.append(clslogit[i,target[i]])
        clslogit = torch.stack(logitlist)

        out = F.conv2d(fms, weight, bias=bias)

        outmaps = []
        for i in range(bs):
            evimap = out[i,target[i]]
            outmaps.append(evimap)

        outmaps = torch.stack(outmaps)
        if imgsize is not None:
            outmaps = outmaps.view(outmaps.size(0),1,outmaps.size(1),outmaps.size(2))
            outmaps = F.interpolate(outmaps,imgsize,mode='bilinear',align_corners=False)

        outmaps = outmaps.squeeze()

        for i in range(bs):
            outmaps[i] -= outmaps[i].min()
            outmaps[i] /= outmaps[i].sum()


    return outmaps,clslogit


def snapmix(input, target, alpha, model=None):

    r = np.random.rand(1)
    lam_a = torch.ones(input.size(0))
    lam_b = 1 - lam_a
    target_b = target.clone()

    if True:
        wfmaps,_ = get_spm(input, target, model)
        bs = input.size(0)
        lam = np.random.beta(alpha, alpha)
        lam1 = np.random.beta(alpha, alpha)
        rand_index = torch.randperm(bs).cuda()
        wfmaps_b = wfmaps[rand_index,:,:]
        target_b = target[rand_index]

        same_label = target == target_b
        bbx1, bby1, bbx2, bby2 = rand_bbox(input.size(), lam)
        bbx1_1, bby1_1, bbx2_1, bby2_1 = rand_bbox(input.size(), lam1)

        area = (bby2-bby1)*(bbx2-bbx1)
        area1 = (bby2_1-bby1_1)*(bbx2_1-bbx1_1)

        if  area1 > 0 and  area>0:
            ncont = input[rand_index, :, bbx1_1:bbx2_1, bby1_1:bby2_1].clone()
            ncont = F.interpolate(ncont, size=(bbx2-bbx1,bby2-bby1), mode='bilinear', align_corners=True)
            input[:, :, bbx1:bbx2, bby1:bby2] = ncont
            lam_a = 1 - wfmaps[:,bbx1:bbx2,bby1:bby2].sum(2).sum(1)/(wfmaps.sum(2).sum(1)+1e-8)
            lam_b = wfmaps_b[:,bbx1_1:bbx2_1,bby1_1:bby2_1].sum(2).sum(1)/(wfmaps_b.sum(2).sum(1)+1e-8)
            tmp = lam_a.clone()
            lam_a[same_label] += lam_b[same_label]
            lam_b[same_label] += tmp[same_label]
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (input.size()[-1] * input.size()[-2]))
            lam_a[torch.isnan(lam_a)] = lam
            lam_b[torch.isnan(lam_b)] = 1-lam

    return input,target,target_b,lam_a.cuda(),lam_b.cuda()

class SnapMixLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, criterion, outputs, ya, yb, lam_a, lam_b):
        loss_a = criterion(outputs, ya)
        loss_b = criterion(outputs, yb)
        loss = torch.mean(loss_a * lam_a + loss_b * lam_b)
        return loss

CutMix

In [27]:
def cutmix(batch, alpha):
    data, targets = batch

    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)

    image_h, image_w = data.shape[2:]
    cx = np.random.uniform(0, image_w)
    cy = np.random.uniform(0, image_h)
    w = image_w * np.sqrt(1 - lam)
    h = image_h * np.sqrt(1 - lam)
    x0 = int(np.round(max(cx - w / 2, 0)))
    x1 = int(np.round(min(cx + w / 2, image_w)))
    y0 = int(np.round(max(cy - h / 2, 0)))
    y1 = int(np.round(min(cy + h / 2, image_h)))

    data[:, :, y0:y1, x0:x1] = shuffled_data[:, :, y0:y1, x0:x1]
    return_targets = torch.zeros((len(targets),3),dtype=torch.int64)
    return_targets[:,0] = targets
    return_targets[:,1] = shuffled_targets
    return_targets[0,2] = lam

    return data, return_targets
        
class CutMixCollator:
    def __init__(self, alpha):
        self.alpha = alpha

    def __call__(self, batch):
        batch = torch.utils.data.dataloader.default_collate(batch)
        batch = cutmix(batch, self.alpha)
        return batch

## Model

In [28]:
# Modified for SnapMix
class CassavaNet(nn.Module):
    def __init__(self):
        super().__init__()
        backbone = timm.create_model(MODEL_ARCH, pretrained=True)
        n_features = backbone.classifier.in_features
        self.backbone = nn.Sequential(*backbone.children())[:-2]
        self.classifier = nn.Linear(n_features, 5)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))

    def forward_features(self, x):
        x = self.backbone(x)
        return x

    def forward(self, x):
        feats = self.forward_features(x)
        x = self.pool(feats).view(x.size(0), -1)
        x = self.classifier(x)
        return x, feats

class CustomEffNet(nn.Module):   
    def __init__(self, model_arch, n_class, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained, n_class)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x
    
class CustomResNext(nn.Module):
    def __init__(self, MODEL_ARCH, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(MODEL_ARCH, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, n_class)

    def forward(self, x):
        x = self.model(x)
        return x

class CustomViT(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, n_class)

    def forward(self, x):
        x = self.model(x)
        return x

class CustomDeiT(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = torch.hub.load('facebookresearch/deit:main', model_arch, pretrained=pretrained)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, n_class)

    def forward(self, x):
        x = self.model(x)
        return x

In [29]:
def GetCriterion(criterion_name, criterion=None):
#     if criterion_name == 'BiTemperedLoss':
#         criterion = BiTemperedLogistic()
#     elif criterion_name == 'SymmetricCrossEntropyLoss':
#         criterion = SymmetricCrossEntropy()
    if criterion_name == 'CrossEntropyLoss':
        criterion = nn.CrossEntropyLoss()
    elif criterion_name == 'LabelSmoothingLoss':
        criterion = LabelSmoothingLoss()
#     elif criterion_name == 'FocalLoss':
#         criterion = FocalLoss()
#     elif criterion_name == 'FocalCosineLoss':
#         criterion = FocalCosineLoss()
    elif criterion_name == 'TaylorCrossEntropyLoss':
        criterion = TaylorCrossEntropyLoss()
    elif criterion_name == 'TaylorSmoothedLoss':
        criterion = TaylorSmoothedLoss()
    elif criterion_name == 'CutMix':
        criterion = CutMixCriterion(criterion)
    elif criterion_name == 'SnapMix':
        criterion = SnapMixLoss()
    return criterion
    
    
def GetScheduler(scheduler_name, optimizer, batches=None):
    #['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts', 'OneCycleLR', 'GradualWarmupSchedulerV2']
    if scheduler_name == 'OneCycleLR':
        return torch.optim.lr_scheduler.OneCycleLR(optimizer,max_lr = 1e-2,epochs = CFG.EPOCHS,
                                                   steps_per_epoch = batches+1,pct_start = 0.1)
    if scheduler_name == 'CosineAnnealingWarmRestarts':
        return torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = T_0, T_mult=1,
                                                                    eta_min=MIN_LR, last_epoch=-1)
    elif scheduler_name == 'CosineAnnealingLR':
        return torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = CFG.T_max, eta_min=0, last_epoch=-1)
    elif scheduler_name == 'ReduceLROnPlateau':
        return torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=1, threshold=0.0001,
                                                          cooldown=0, min_lr=MIN_LR)
#     elif scheduler_name == 'GradualWarmupSchedulerV2':
#         return GradualWarmupSchedulerV2(optimizer=optimizer)
    
def GetOptimizer(optimizer_name,parameters):
    #['Adam','Ranger']
    if optimizer_name == 'Adam':
#         if CFG.scheduler_name == 'GradualWarmupSchedulerV2':
#             return torch.optim.Adam(parameters, lr=CFG.LR_START, weight_decay=CFG.weight_decay, amsgrad=False)
#         else:
        return torch.optim.Adam(parameters, lr=LR, weight_decay=WEIGHT_DECAY, amsgrad=False)
    elif optimizer_name == 'AdamW':
#         if CFG.scheduler_name == 'GradualWarmupSchedulerV2':
#             return torch.optim.AdamW(parameters, lr=CFG.LR_START, weight_decay=CFG.weight_decay, amsgrad=False)
#         else:
        return torch.optim.Adam(parameters, lr=LR, weight_decay=WEIGHT_DECAY, amsgrad=False)
    elif optimizer_name == 'AdamP':
#         if CFG.scheduler_name == 'GradualWarmupSchedulerV2':
#             return AdamP(parameters, lr=CFG.LR_START, weight_decay=CFG.weight_decay)
#         else:
        return AdamP(parameters, lr=LR, weight_decay=WEIGHT_DECAY)
    elif optimizer_name == 'Ranger':
        return Ranger(parameters, lr = LR, alpha = 0.5, k = 6, N_sma_threshhold = 5, 
                      betas = (0.95,0.999), weight_decay=WEIGHT_DECAY)
    elif optimizer_name == 'SAM':
        return SAM(parameters, BASE_OPTIMIZER, lr=0.1, momentum=0.9,weight_decay=0.0005)

## Train Function

In [33]:
def train_fn(model, dataloader, device, epoch, optimizer, criterion, scheduler):
    
    data_time = AverageMeter()
    batch_time = AverageMeter()
    losses = AverageMeter()
    accuracies = AverageMeter()
    
    model.train()
    scaler = GradScaler()
    start_time = time.time()
    loader = tqdm(dataloader, total=len(dataloader))
    for step, (images, labels) in enumerate(loader):
        
        images = images.to(device)
        labels = labels.to(device)
        data_time.update(time.time() - start_time)
        
        with autocast():
            
            if SNAPMIX:
                rand = np.random.rand()
                if rand > (1.0-SNAPMIX_PCT):
                    X, ya, yb, lam_a, lam_b = snapmix(images, labels, SNAPMIX_ALPHA, model)
                    output, _ = model(X)
                    snapmix_criterion = GetCriterion('SnapMix')
                    loss = snapmix_criterion(criterion, output, ya, yb, lam_a, lam_b)
                else:
                    output, _ = model(images)
                    criterion = GetCriterion('CrossEntropyLoss').to(device)
                    loss = torch.mean(criterion(output, labels))
                losses.update(loss.item(), BATCH_SIZE)
                
            else:
                output = model(images)
                loss = criterion(output, labels)
                if CUTMIX:
                    losses.update(loss, BATCH_SIZE)
                else:
                    losses.update(loss.item(), BATCH_SIZE)
            
        scaler.scale(loss).backward()
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm = MAX_NORM)

        if not CUTMIX:
                accuracy = (output.argmax(dim=1) == labels).float().mean()
        else:
            if labels[0,2] >= 0.5:
                accuracy = (output.argmax(dim=1) == labels[:,0]).float().mean()
            else:
                accuracy = (output.argmax(dim=1) == labels[:,1]).float().mean()

        accuracies.update(accuracy.item(), BATCH_SIZE)
        
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        
        if scheduler is not None and SCHEDULER_UPDATE == 'batch':
            scheduler.step()

        batch_time.update(time.time() - start_time)
        start_time = time.time()
        
        if step % ITER_FREQ == 0:
            
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f}s ({batch_time.avg:.3f}s), '
                  'Data Time {data_time.val:.3f}s ({data_time.avg:.3f}s)\t'
                  'Loss: {loss.val:.4f} ({loss.avg:.4f}), '
                  'Accuracy {accuracy.val:.4f} ({accuracy.avg:.4f})'.format((epoch+1),
                                                                             step, len(dataloader),
                                                                             batch_time=batch_time,
                                                                             data_time=data_time,
                                                                             loss=losses,
                                                                             accuracy=accuracies))
        loader.set_postfix(loss=losses.avg, accuracy=accuracies.avg)
    if scheduler is not None and SCHEDULER_UPDATE == 'epoch':
        scheduler.step()
        
    return losses.avg, accuracies.avg

## Main

In [34]:
def engine(device, model_path=None):
    
    train_data = CLD_train_DS(df, transform=get_transform())

    if CUTMIX:
        print('Using CutMix')
        collator = CutMixCollator(CM_ALPHA)
        criterion = GetCriterion('CutMix', GetCriterion(CRITERION)).to(device)                
    
    train_loader = DataLoader(train_data, 
                              batch_size=BATCH_SIZE, 
                              shuffle=True, 
                              num_workers=NUM_WORKERS,
#                               collate_fn=collator,   # Uncomment when using CutMix
                              pin_memory=True, 
                              drop_last=True)
    
    batches = len(train_loader)
    if model_path is not None:
        model = torch.load(model_path)
        START_EPOCH = int(model_path.split('_')[-1])
    else:
        model = CassavaNet()
        START_EPOCH = 0
    model.to(device)
    
    params = filter(lambda p: p.requires_grad, model.parameters())    
    optimizer = GetOptimizer(OPTIMIZER, params)
    if not CUTMIX:
        criterion = GetCriterion(CRITERION).to(device)
    else:
        criterion = GetCriterion('CutMix', GetCriterion(CRITERION)).to(device)
    scheduler = GetScheduler(SCHEDULER, optimizer)
    
    loss = []
    accuracy = []
    for epoch in range(START_EPOCH, EPOCHS):
        
        epoch_start = time.time()        
        avg_loss, avg_accuracy = train_fn(model, train_loader, device, epoch, optimizer, criterion, scheduler)
        epoch_end = time.time() - epoch_start
        
        loss.append(avg_loss)
        accuracy.append(avg_accuracy)
        
        content = f'Epoch {epoch+1} - avg_loss: {avg_loss:.4f} avg_accuracy: {avg_accuracy:.4f} time: {epoch_end:.0f}s'
        with open(f'GPU_{MODEL_ARCH}_{OPTIMIZER}_{CRITERION}.txt', 'a') as appender:
            appender.write(content + '\n')
        
        # Save the model to use it for inference.
        torch.save(model.state_dict(), f'GPU_{MODEL_ARCH}_epoch_{(epoch+1)}.pth')
        torch.save(model, f'GPU_{MODEL_ARCH}_epoch_{(epoch+1)}')
        torch.cuda.empty_cache()
    
    return {'loss':loss, 'accuracy':accuracy}

In [ ]:
logs = engine(DEVICE)

  0%|          | 0/1646 [00:00<?, ?it/s]

Epoch: [1][0/1646]	Batch Time 5.094s (5.094s), Data Time 3.655s (3.655s)	Loss: 2.2262 (2.2262), Accuracy 0.1875 (0.1875)
Epoch: [1][500/1646]	Batch Time 0.579s (0.584s), Data Time 0.008s (0.018s)	Loss: 1.3033 (1.1869), Accuracy 0.7500 (0.6831)
Epoch: [1][1000/1646]	Batch Time 0.676s (0.579s), Data Time 0.008s (0.014s)	Loss: 0.8196 (1.1104), Accuracy 0.6875 (0.7120)
Epoch: [1][1500/1646]	Batch Time 0.608s (0.577s), Data Time 0.009s (0.013s)	Loss: 1.0750 (1.0722), Accuracy 0.8750 (0.7260)


  0%|          | 0/1646 [00:00<?, ?it/s]

Epoch: [2][0/1646]	Batch Time 6.124s (6.124s), Data Time 3.416s (3.416s)	Loss: 0.4640 (0.4640), Accuracy 0.8125 (0.8125)
Epoch: [2][500/1646]	Batch Time 0.623s (0.586s), Data Time 0.021s (0.017s)	Loss: 1.8323 (0.9681), Accuracy 0.5000 (0.7672)
Epoch: [2][1000/1646]	Batch Time 0.569s (0.580s), Data Time 0.009s (0.014s)	Loss: 1.3079 (0.9668), Accuracy 0.8750 (0.7690)
